In [5]:
import pandas as pd
pd.options.display.max_columns = None
import re
import os
import time
from tqdm import tqdm
import opendatasets as od
import pandas
# from dataset_utilities import value_counts_plus

for dirname, _, filenames in os.walk('C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

C:\Users\user\Documents\LogfileJuypter\regressionTree\access.log
C:\Users\user\Documents\LogfileJuypter\regressionTree\access10.log
C:\Users\user\Documents\LogfileJuypter\regressionTree\deploy.py
C:\Users\user\Documents\LogfileJuypter\regressionTree\labeledData.csv
C:\Users\user\Documents\LogfileJuypter\regressionTree\labeledDataModified.csv
C:\Users\user\Documents\LogfileJuypter\regressionTree\logfile-cleaning_modified.ipynb
C:\Users\user\Documents\LogfileJuypter\regressionTree\logs_df.parquet
C:\Users\user\Documents\LogfileJuypter\regressionTree\logToCSVscript.py
C:\Users\user\Documents\LogfileJuypter\regressionTree\RegressionTreesLogAnalysis.ipynb
C:\Users\user\Documents\LogfileJuypter\regressionTree\savemodel.sav
C:\Users\user\Documents\LogfileJuypter\regressionTree\Test.csv
C:\Users\user\Documents\LogfileJuypter\regressionTree\.ipynb_checkpoints\logfile-cleaning_modified-checkpoint.ipynb
C:\Users\user\Documents\LogfileJuypter\regressionTree\.ipynb_checkpoints\Log_file_Cleaning-che

In [6]:
# There is a minor bug in this regex, it misses the last field. I'll fix this soon. 

common_regex = '^(?P<ip>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-)'
combined_regex = '^(?P<ip>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<browser>[^"]*)'
columns = ['ip', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'browser']

In [7]:
import time
import re
import pandas as pd


def logs_to_df(logfile, output_dir, errors_file):
    with open(logfile) as source_file:
        linenumber = 0
        parsed_lines = []
        for line in tqdm(source_file):
            try:
                log_line = re.findall(combined_regex, line)[0]
                parsed_lines.append(log_line)
            except Exception as e:
                with open(errors_file, 'at') as errfile:
                    print((line, str(e)), file=errfile)
                continue
            linenumber += 1
            if linenumber % 250_000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear()
        else:
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
            parsed_lines.clear()

In [9]:
# %%time
logs_to_df(logfile=r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\templates\\uploads\\access10.log', output_dir=r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\parquet_dir', errors_file=r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\error.txt')

118it [00:00, 19711.96it/s]


In [10]:
# %%time 
logs_df = pd.read_parquet(r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\parquet_dir')
# %rm -r r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\parquet_dir\\'
# logs_df.info(show_counts=True, verbose=True)

In [11]:
logs_df['ip'] = logs_df['ip'].astype('category')
del logs_df['userid']
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], format='%d/%b/%Y:%H:%M:%S %z')
logs_df['method'] = logs_df['method'].astype('category')
logs_df['status'] = logs_df['status'].astype('int16')
logs_df['size'] = logs_df['size'].astype('int32')
logs_df['referer'] = logs_df['referer'].astype('category')
logs_df['browser'] = logs_df['browser'].astype('category')

In [12]:
# %%time 
logs_df.to_parquet('logs_df.parquet')

In [13]:
# %%time 
logs_df = pd.read_parquet('logs_df.parquet')

In [14]:
# arrtoex = pd.DataFrame(logs_df).T
ArrtoExSliced = logs_df.iloc[:10000,0:]
# ArrtoExSliced.to_excel(excel_writer = r"C:\Users\jayes\OneDrive\Desktop\CSV and Datasets\Output\convaccess.xlsx\\")
ArrtoExSliced

,ip,datetime,method,request,status,size,referer,browser
0,40.77.167.129,2019-01-22 03:56:19+03:30,GET,/image/6248/productModel/150x150,200,2788,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
1,40.77.167.129,2019-01-22 03:56:20+03:30,GET,/image/64815/productModel/150x150,200,3481,-,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
2,66.249.66.194,2019-01-22 03:56:20+03:30,GET,"/m/filter/b2,p6",200,19451,-,Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Bu...
3,91.99.72.15,2019-01-22 03:56:20+03:30,GET,/product/32798/63266/%DB%8C%D8%AE%DA%86%D8%A7%...,200,40250,-,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...
4,178.253.33.51,2019-01-22 03:56:20+03:30,GET,/settings/logo,200,4120,https://www.zanbil.ir/m/product/32574/62991/%D...,Mozilla/5.0 (Linux; Android 5.1; HTC Desire 72...
...,...,...,...,...,...,...,...,...
111,66.111.54.249,2019-01-22 03:56:51+03:30,GET,/image/64431/productModel/200x200,200,4227,https://www.zanbil.ir/m/browse/refrigerator-an...,Mozilla/5.0 (Linux; Android 5.0; SM-G900H Buil...
112,66.111.54.249,2019-01-22 03:56:51+03:30,GET,/image/65362/productModel/200x200,200,3776,https://www.zanbil.ir/m/browse/refrigerator-an...,Mozilla/5.0 (Linux; Android 5.0; SM-G900H Buil...
113,173.249.54.67,2019-01-22 03:56:51+03:30,GET,/image/34185?name=mfp-m130a-1.jpg&wh=200x200,200,5395,-,Dalvik/2.1.0 (Linux; U; Android 8.0.0; PRA-LA1...
114,91.99.72.15,2019-01-22 03:56:52+03:30,GET,/product/2623?model=2699,200,43555,-,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...


In [15]:
arrtoex = pd.DataFrame(ArrtoExSliced).T

In [16]:
colarr = ['ip', 'datetime', 'method', 'request', 'status', 'size', 'referer', '']
import xlsxwriter
workbook = xlsxwriter.Workbook(r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\CSVlogshort.xlsx')
worksheet = workbook.add_worksheet()
for i in range(0,8):
     worksheet.write(0,i,str(colarr[i]))
workbook.close()

In [17]:
import xlsxwriter
workbook = xlsxwriter.Workbook(r'C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\CSVlogshort.xlsx')
row = 0
colarr = ['Client', 'DateTime', 'Method', 'Request', 'Status', 'Size', 'Referer', 'browser']
worksheet = workbook.add_worksheet()
for i in range(0,8):
     worksheet.write(0,i,str(colarr[i]))
for i in range(1,len(ArrtoExSliced)):
     for j in range(0,8):
        worksheet.write(i,j,str(arrtoex[i][j]))
workbook.close()

In [18]:
# dff = pd.read_csv('C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\CSVlogshort.csv')
import openpyxl
read_file = pd.DataFrame(pd.read_excel("output\\CSVlogshort.xlsx"))

read_file.to_csv ("C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\Test.csv", 
                  index = None,
                  header=True)

df = pd.DataFrame(pd.read_csv("Test.csv"))
  
# show the dataframe
# df
import fileinput
# for line in fileinput.input('C:\\Users\\user\\Documents\\LogfileJuypter\\regressionTree\\output\\Test.csv', inplace=True):
#     if fileinput.isfirstline():
#         print( 'ip,datetime, method,request,status,size,referer,browser')
#     else:
#         print (line)